In [ ]:
# AWS Deployment Steps:

import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

!mkdir keras_model
# Upload files then ls:
!ls keras_model

from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils 
import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Note: This directory structure will need to be followed - see notes 
# for the next section
model_version = '1'
export_dir = 'export/Servo/' + model_version

import shutil
shutil.rmtree(export_dir)

# Build the Protocol Buffer SavedModel at 'export_dir'
build = builder.SavedModelBuilder(export_dir)

import os
import numpy as np
import cv2
from numpy import dot
from numpy.linalg import norm
import datetime
import keras

loaded_model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, pooling='max',input_shape=(535,400,3))

# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, \
    outputs={"score": loaded_model.output})

from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    build.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], \
        signature_def_map={"serving_default": signature})
    build.save()
    
!ls export
!ls export/Servo
!ls export/Servo/1/variables

import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)
    
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', \
                key_prefix='model')

!touch train.py

from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + \
    sagemaker_session.default_bucket() + '/model/model.tar.gz',
    role = role,
    framework_version = '1.12',
    entry_point = 'train.py')

%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

#notebook instance: ml.t2.medium

print(predictor.endpoint)

In [ ]:
# change name as per previous cell output:
# Approach 1: load deployed model
endpoint_name = 'sagemaker-tensorflow-2019-08-05-03-29-25-591'

import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(\
        endpoint_name, sagemaker_session)

In [ ]:
# Approach 1: predict using deployed model on AWS sagemaker:
###feature extractor of images in database
import os
import numpy as np
import cv2
from numpy import dot
from numpy.linalg import norm
import datetime

features_list = []
fname=[]
for files in os.listdir('processed_input'):
    fname.append(files)
    print(files)
    img = cv2.imread('processed_input//'+files)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(400,535))
    img = np.array(img) 
    img = img/255 
    img = np.expand_dims(img, axis=0)
    intermediate_prediction=predictor.predict(img)[0]
    features_list.append(intermediate_prediction.flatten())

In [2]:
# Saving weights with JSON architecture:
# import tensorflow as tf
import keras
loaded_model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, pooling='max',input_shape=(535,400,3))
# serialize model to JSON
model_json = loaded_model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
loaded_model.save_weights("model1.h5")
print("Saved model to disk")

Using TensorFlow backend.


Saved model to disk


In [1]:
# Saving standalone H5 weight file:
# import tensorflow as tf
import keras
loaded_model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, pooling='max',input_shape=(535,400,3))
loaded_model.save("model-standalone.h5")
print("Saved model to disk")

Using TensorFlow backend.


Saved model to disk


In [3]:
# Approach 2: Load save model from disk to compare:
from keras.models import model_from_json
json_file = open('model1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights('model1.h5')
print("Loaded model from disk")

Loaded model from disk


In [6]:
#Approach 2: Predict from model loaded from disk:
###feature extractor of images in database
import os
import numpy as np
import cv2
from numpy import dot
from numpy.linalg import norm
import datetime

features_list = []
fname=[]
for files in os.listdir('processed_input'):
    fname.append(files)
    print(files)
    img = cv2.imread('processed_input//'+files)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(400,535))
    img = np.array(img) 
    img = img/255 
    img = np.expand_dims(img, axis=0)
    intermediate_prediction=loaded_model.predict(img)[0]
    features_list.append(intermediate_prediction.flatten())

CL_Book1_Pg4.png
CL_Book1_Pg5.png
CL_Book1_Pg7.png
CL_Book1_Pg6.png
CL_Book1_Pg2.png
CL_Book1_Pg3.png
CL_Book1_Pg1.png
CL_Book1_Pg0.png
NCERT_Pg3.png
CL_Book1_Pg14.png
CL_Book1_Pg15.png
NCERT_Pg2.png
NCERT_Pg0.png
CL_Book1_Pg17.png
CL_Book1_Pg16.png
NCERT_Pg1.png
CL_Book1_Pg12.png
CL_Book1_Pg13.png
NCERT_Pg4.png
CL_Book1_Pg11.png
CL_Book1_Pg8.png
CL_Book1_Pg9.png
CL_Book1_Pg10.png


In [7]:
###feature extractor for query image and comparing it against every feature_vec in db
import tensorflow as tf
import os
import datetime
import cv2
import numpy as np
from numpy import dot
from numpy.linalg import norm

f_list = sorted(os.listdir('test'))
start1 = datetime.datetime.now()

for files in f_list:  ###list of queries(test images)
    img = cv2.imread('test/'+files)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(400,535))
    #img, alpha, beta = automatic_brightness_and_contrast(img)
    img = np.array(img) 
    img = img/255
    img = np.expand_dims(img, axis=0)
    x=loaded_model.predict(img)[0]
    min = float('inf') 
    for i in range(len(features_list)): ###list of features in db
        a = x.flatten()
        b= features_list[i]
        cos = dot(a, b)/(norm(a)*norm(b))
        val = np.linalg.norm(a-b)
        if(val<min):
            min = val
            can = fname[i]
            cos_f = cos
    print('input ',files,'output ',can,'ED = ',min,'cosine = ',cos_f)
    end1 = datetime.datetime.now()
print('  complete process took', round((end1-start1).total_seconds(), 2), 'sec.') 


input  CL_Book1_Pg0.png output  CL_Book1_Pg0.png ED =  0.3874467 cosine =  0.99981725
  complete process took 1.7 sec.
